In [99]:
import keras.backend as K
from keras.layers import Input, Dense, Concatenate, Flatten, Dropout, Conv3D, MaxPooling3D, ZeroPadding3D
from keras.models import Model
from keras.regularizers import l1, l2
from keras.optimizers import Adam
from keras.constraints import max_norm
from keras.layers.noise import GaussianNoise
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

import config
import copy
import csv
import helper_fxns as hf
import importlib
import math
import matplotlib.pyplot as plt
import numpy as np
import operator
import os
import pandas as pd
import random
import re
from scipy.misc import imsave
import transforms as tr
import tensorflow as tf
%matplotlib inline

In [100]:
importlib.reload(config)
C = config.Config()

In [101]:
classes_to_include = ['cyst', 'hcc', 'hemangioma', 'cholangio', 'colorectal']#, 'fnh']
dims = C.dims
nb_classes = len(classes_to_include)

voi_img = Input(shape=(dims[0], dims[1], dims[2], C.nb_channels))
x = voi_img
#x = GaussianNoise(1)(x)
#x = ZeroPadding3D(padding=(3,3,2))(voi_img)
x = Conv3D(filters=128, kernel_size=(3,3,2), activation='relu')(x)
x = Dropout(0.5)(x)
x = Conv3D(filters=128, kernel_size=(3,3,2), activation='relu')(x)
x = MaxPooling3D((2, 2, 2))(x)
x = Dropout(0.5)(x)
#x = Conv3D(filters=64, kernel_size=(3,3,2), strides=(2, 2, 2), activation='relu', kernel_constraint=max_norm(4.))(x)
#x = Dropout(0.5)(x)
x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(x)
x = MaxPooling3D((2, 2, 1))(x)
x = Dropout(0.5)(x)
x = Flatten()(x)

img_traits = Input(shape=(2,)) #bounding volume and aspect ratio of lesion

intermed = Concatenate(axis=1)([x, img_traits])
x = Dense(64, activation='relu')(intermed)#, kernel_initializer='normal', kernel_regularizer=l1(.01), kernel_constraint=max_norm(3.))(x)
x = Dropout(0.5)(x)
pred_class = Dense(nb_classes, activation='softmax')(x)#Dense(nb_classes, activation='softmax')(x)

In [116]:
classes_to_include = ['cyst', 'hcc', 'hemangioma', 'cholangio', 'colorectal']#, 'fnh']
dims = C.dims
nb_classes = len(classes_to_include)

art_img = Input(shape=(dims[0], dims[1], dims[2], 1))
art_x = art_img
art_x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(art_x)
art_x = Dropout(0.5)(art_x)
art_x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(art_x)
art_x = MaxPooling3D((2, 2, 2))(art_x)
art_x = Dropout(0.5)(art_x)
art_x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(art_x)
art_x = MaxPooling3D((2, 2, 1))(art_x)
art_x = Dropout(0.5)(art_x)
art_x = Flatten()(art_x)

ven_img = Input(shape=(dims[0], dims[1], dims[2], 1))
ven_x = ven_img
ven_x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(ven_x)
ven_x = Dropout(0.5)(ven_x)
ven_x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(ven_x)
ven_x = MaxPooling3D((2, 2, 2))(ven_x)
ven_x = Dropout(0.5)(ven_x)
ven_x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(ven_x)
ven_x = MaxPooling3D((2, 2, 1))(ven_x)
ven_x = Dropout(0.5)(ven_x)
ven_x = Flatten()(ven_x)

eq_img = Input(shape=(dims[0], dims[1], dims[2], 1))
eq_x = eq_img
eq_x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(eq_x)
eq_x = Dropout(0.5)(eq_x)
eq_x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(eq_x)
eq_x = MaxPooling3D((2, 2, 2))(eq_x)
eq_x = Dropout(0.5)(eq_x)
eq_x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(eq_x)
eq_x = MaxPooling3D((2, 2, 1))(eq_x)
eq_x = Dropout(0.5)(eq_x)
eq_x = Flatten()(eq_x)

img_traits = Input(shape=(2,)) #bounding volume and aspect ratio of lesion

intermed = Concatenate(axis=1)([art_x, ven_x, eq_x, img_traits])
x = Dense(64, activation='relu')(intermed)#, kernel_initializer='normal', kernel_regularizer=l1(.01), kernel_constraint=max_norm(3.))(x)
x = Dropout(0.5)(x)
pred_class = Dense(nb_classes, activation='softmax')(x)#Dense(nb_classes, activation='softmax')(x)

In [117]:
optim = Adam(lr=0.01)#5, decay=0.001)
early_stopping = EarlyStopping(monitor='acc', min_delta=0.01, patience=4)

model = Model([art_img, ven_img, eq_img, img_traits], pred_class)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_28 (InputLayer)            (None, 36, 36, 12, 1) 0                                            
____________________________________________________________________________________________________
input_29 (InputLayer)            (None, 36, 36, 12, 1) 0                                            
____________________________________________________________________________________________________
input_30 (InputLayer)            (None, 36, 36, 12, 1) 0                                            
____________________________________________________________________________________________________
conv3d_42 (Conv3D)               (None, 34, 34, 11, 64 1216        input_28[0][0]                   
___________________________________________________________________________________________

In [ ]:
optim = Adam(lr=0.01)#5, decay=0.001)
early_stopping = EarlyStopping(monitor='acc', min_delta=0.01, patience=4)

model = Model([voi_img, img_traits], pred_class)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

model2 = Model(voi_img, intermed)

for l in range(2,len(model2.layers)):
    model2.layers[l].set_weights(model.layers[l].get_weights())

In [103]:
orig_data_dict = {}
num_samples = {}
voi_df = pd.read_csv(C.art_voi_path)
intensity_df = pd.read_csv(C.int_df_path)
#intensity_df.loc[intensity_df["art_int"] == 0, "art_int"] = np.mean(intensity_df[intensity_df["art_int"] > 0]["art_int"])
#intensity_df.loc[intensity_df["ven_int"] == 0, "ven_int"] = np.mean(intensity_df[intensity_df["ven_int"] > 0]["ven_int"])
#intensity_df.loc[intensity_df["eq_int"] == 0, "eq_int"] = np.mean(intensity_df[intensity_df["eq_int"] > 0]["eq_int"])

def rescale_int(img, intensity_row):
    img[:,:,:,0] = img[:,:,:,0] / float(intensity_row["art_int"])
    img[:,:,:,1] = img[:,:,:,1] / float(intensity_row["ven_int"])
    img[:,:,:,2] = img[:,:,:,2] / float(intensity_row["eq_int"])
    
    return img

def collect_unaug_data(classes_to_include, C):
    """Return dictionary pointing to X (img data) and Z (filenames) and dictionary storing number of samples of each class."""
    
    for class_name in classes_to_include:
        x = np.empty((10000, C.dims[0], C.dims[1], C.dims[2], C.nb_channels))
        x2 = np.empty((10000, 2))
        z = []

        for index, img_fn in enumerate(os.listdir(C.orig_dir+class_name)):
            try:
                x[index] = np.load(C.orig_dir+class_name+"\\"+img_fn)
            except:
                raise ValueError(C.orig_dir+class_name+"\\"+img_fn + " not found")
            z.append(img_fn)
            
            row = voi_df[(voi_df["Filename"] == img_fn[:img_fn.find('_')] + ".npy") &
                         (voi_df["lesion_num"] == int(img_fn[img_fn.find('_')+1:-4]))]
            
            try:
                x2[index] = [(float(row["real_dx"]) * float(row["real_dy"]) * float(row["real_dz"])) ** (1/3) / 50,
                            max(float(row["real_dx"]), float(row["real_dy"])) / float(row["real_dz"])]
            except TypeError:
                raise ValueError(img_fn + " is probably missing a voi_df entry.")
            
            x[index] = rescale_int(x[index], intensity_df[intensity_df["AccNum"] == img_fn[:img_fn.find('_')]])

        x.resize((index, dims[0], dims[1], dims[2], C.nb_channels)) #shrink first dimension to fit
        x2.resize((index, 2)) #shrink first dimension to fit
        orig_data_dict[class_name] = [x,x2,np.array(z)]
        num_samples[class_name] = index
        
    return orig_data_dict, num_samples

In [106]:
orig_data_dict, num_samples = collect_unaug_data(classes_to_include, C)
print(num_samples)

{'colorectal': 70, 'hemangioma': 60, 'hcc': 55, 'cyst': 77, 'cholangio': 61}


In [107]:
train_ids = {} #filenames of training set originals
test_ids = {} #filenames of test set
X_test = []
X2_test = []
Y_test = []
Z_test = []
X_train_orig = []
X2_train_orig = []
Y_train_orig = []
Z_train_orig = []
cls_mapping = []

train_samples = {}

for cls_num, cls in enumerate(orig_data_dict):
    cls_mapping.append(cls)
    
    train_samples[cls] = round(num_samples[cls]*C.train_frac)
    
    order = np.random.permutation(list(range(num_samples[cls])))
    train_ids[cls] = list(orig_data_dict[cls][2][order[:train_samples[cls]]])
    test_ids[cls] = list(orig_data_dict[cls][2][order[train_samples[cls]:]])
    
    X_test = X_test + list(orig_data_dict[cls][0][order[train_samples[cls]:]])
    X2_test = X2_test + list(orig_data_dict[cls][1][order[train_samples[cls]:]])
    Y_test = Y_test + [[0] * cls_num + [1] + [0] * (nb_classes - cls_num - 1)] * \
                        (num_samples[cls] - train_samples[cls])
    Z_test = Z_test + test_ids[cls]
    
    X_train_orig = X_train_orig + list(orig_data_dict[cls][0][order[:train_samples[cls]]])
    X2_train_orig = X2_train_orig + list(orig_data_dict[cls][1][order[:train_samples[cls]]])
    Y_train_orig = Y_train_orig + [[0] * cls_num + [1] + [0] * (nb_classes - cls_num - 1)] * \
                        (train_samples[cls])
    Z_train_orig = Z_train_orig + train_ids[cls]
    
    print("%s has %d samples for training (%d after augmentation) and %d for testing" %
          (cls, train_samples[cls], train_samples[cls] * C.aug_factor, num_samples[cls] - train_samples[cls]))
    
#Y_test = np_utils.to_categorical(Y_test, nb_classes)
#Y_train_orig = np_utils.to_categorical(Y_train_orig, nb_classes)

colorectal has 56 samples for training (2800 after augmentation) and 14 for testing
hemangioma has 48 samples for training (2400 after augmentation) and 12 for testing
hcc has 44 samples for training (2200 after augmentation) and 11 for testing
cyst has 62 samples for training (3100 after augmentation) and 15 for testing
cholangio has 49 samples for training (2450 after augmentation) and 12 for testing


def scale_to_one(X, scale_by_channel=True):
    def scale_img(x):
        return 2 * (x - np.amin(x)) / (np.amax(x) - np.amin(x)) - 1
    return np.array(X)

    X = np.array(X)
    
    if scale_by_channel:
        X = [np.stack([scale_img(x[:,:,:,0]), scale_img(x[:,:,:,1]), scale_img(x[:,:,:,2])], axis=3) for x in X]
    else:
        X = [scale_img(x) for x in X]
    
    return np.array(X)

In [108]:
#scale from -1 to 1
#X_train = [scale_to_one(X_train), tbd(X2_train)]
#X_val = [scale_to_one(X_val), tbd(X2_val)]
X_test = [np.array(X_test), np.array(X2_test)]
X_train_orig = [np.array(X_train_orig), np.array(X2_train_orig)]

#Y_train = np.array(Y_train)
#Y_val = np.array(Y_val)
Y_test = np.array(Y_test)
Y_train_orig = np.array(Y_train_orig)

#Z_train = np.array(Z_train)
Z_test = np.array(Z_test)
Z_train_orig = np.array(Z_train_orig)

In [109]:
avg_X2 = {}
for cls in classes_to_include:
    avg_X2[cls] = np.mean(orig_data_dict[cls][1], axis=0)

In [110]:
def train_generator_func(n=12, n_art=0):
    """n is the number of samples from each class, n_art is the number of artificial samples"""
    
    num_classes = len(classes_to_include)
    while True:
        x1 = np.empty(((n+n_art)*num_classes, dims[0], dims[1], dims[2], C.nb_channels))
        x2 = np.empty(((n+n_art)*num_classes, 2))
        y = np.zeros(((n+n_art)*num_classes, len(classes_to_include)))

        train_cnt = 0
        for cls in classes_to_include:
            img_fns = os.listdir(C.aug_dir+cls)
            while True:
                img_fn = random.choice(img_fns)
                if img_fn[:img_fn.rfind('_')] + ".npy" in train_ids[cls]:
                    x1[train_cnt] = np.load(C.aug_dir+cls+"\\"+img_fn)
                    x1[train_cnt] = rescale_int(x1[train_cnt],
                                          intensity_df[intensity_df["AccNum"] == img_fn[:img_fn.find('_')]])

                    row = voi_df[(voi_df["Filename"] == img_fn[:img_fn.find('_')] + ".npy") &
                                 (voi_df["lesion_num"] == int(img_fn[img_fn.find('_')+1:img_fn.rfind('_')]))]
                    x2[train_cnt] = [(float(row["real_dx"]) * float(row["real_dy"]) * float(row["real_dz"])) ** (1/3) / 50,
                                        max(float(row["real_dx"]), float(row["real_dy"])) / float(row["real_dz"])]
                    
                    y[train_cnt][cls_mapping.index(cls)] = 1
                    
                    train_cnt += 1
                    if train_cnt % n == 0:
                        break
            
                        
            img_fns = os.listdir(C.artif_dir+cls)
            for _ in range(n_art):
                img_fn = random.choice(img_fns)
                x1[train_cnt] = np.load(C.artif_dir + cls + "\\" + img_fn)
                x2[train_cnt] = avg_X2[cls]
                y[train_cnt][cls_mapping.index(cls)] = 1

                train_cnt += 1
        
        yield [np.array(x1), np.array(x2)], np.array(y)

In [111]:
train_generator = train_generator_func()
model.fit_generator(train_generator, steps_per_epoch=100, epochs=40, callbacks=[early_stopping])

Epoch 1/40


Exception in thread Thread-23:
Traceback (most recent call last):
  File "C:\Users\Clinton\AppData\Local\conda\conda\envs\old-keras\lib\threading.py", line 923, in _bootstrap_inner
    self.run()
  File "C:\Users\Clinton\AppData\Local\conda\conda\envs\old-keras\lib\threading.py", line 871, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Clinton\AppData\Local\conda\conda\envs\old-keras\lib\site-packages\keras\utils\data_utils.py", line 560, in data_generator_task
    generator_output = next(self._generator)
  File "<ipython-input-110-d19a3d22f5e4>", line 16, in train_generator_func
    x1[train_cnt] = np.load(C.aug_dir+cls+"\\"+img_fn)
ValueError: could not broadcast input array from shape (35,36,12,3) into shape (36,36,12,3)



StopIteration: 

hist = model.fit(X_train2, Y_train2, batch_size=32, epochs=200, validation_data=(X_test, Y_test), callbacks=[early_stopping])

In [66]:
with open(C.small_voi_path, 'r') as csv_file:
    reader = csv.reader(csv_file)
    small_vois = dict(reader)
    
for key in small_vois:
    small_vois[key] = [int(x) for x in small_vois[key][1:-1].split(', ')]

In [67]:
voi_df_art = pd.read_csv(C.art_voi_path)
voi_df_ven = pd.read_csv(C.ven_voi_path)
voi_df_eq = pd.read_csv(C.eq_voi_path)

In [68]:
def save_output(Z, y_pred, cls_mapping):
    save_dir = C.output_img_dir
    for cls in classes_to_include:
        if not os.path.exists(save_dir + "\\correct\\" + cls):
            os.makedirs(save_dir + "\\correct\\" + cls)
        if not os.path.exists(save_dir + "\\incorrect\\" + cls):
            os.makedirs(save_dir + "\\incorrect\\" + cls)

    for i in range(len(Z)):
        if y_pred[i] != y_true[i]:
            plot_multich_with_bbox(Z[i], cls_mapping[y_pred[i]], save_dir=save_dir + "\\incorrect\\" + cls_mapping[y_true[i]])
        else:
            plot_multich_with_bbox(Z[i], cls_mapping[y_pred[i]], save_dir=save_dir + "\\correct\\" + cls_mapping[y_true[i]])  

In [69]:
def draw_bbox(img_slice, final_dims, voi):
    x1 = voi[0]
    x2 = voi[1]
    y1 = voi[2]
    y2 = voi[3]
    z1 = voi[4]
    z2 = voi[5]
    dx = x2 - x1
    dy = y2 - y1
    dz = z2 - z1
    
    buffer = 0.85
    scale_ratios = [final_dims[0]/dx * buffer, final_dims[1]/dy * buffer, final_dims[2]/dz * buffer]
    
    crop = [img_slice.shape[i] - round(final_dims[i]/scale_ratios[i]) for i in range(2)]
    
    for i in range(2):
        assert crop[i]>=0
        
    x1=crop[0]//2
    x2=-crop[0]//2
    y1=crop[1]//2
    y2=-crop[1]//2
        
    img_slice[x1:x2, y2, 2, :] = 255
    img_slice[x1:x2, y2, :2, :] = 0

    img_slice[x1:x2, y1, 2, :] = 255
    img_slice[x1:x2, y1, :2, :] = 0

    img_slice[x1, y1:y2, 2, :] = 255
    img_slice[x1, y1:y2, :2, :] = 0

    img_slice[x2, y1:y2, 2, :] = 255
    img_slice[x2, y1:y2, :2, :] = 0
    
    return img_slice

In [70]:
def plot_multich_with_bbox(fn, pred_class, num_ch=3, save_dir=None):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    img_fn = fn[:fn.find('_')] + ".npy"
    voi = voi_df_art[(voi_df_art["Filename"] == img_fn) &
                     (voi_df_art["lesion_num"] == int(fn[fn.find('_')+1:fn.rfind('.')]))].iloc[0]
    
    img = np.load(C.crops_dir + voi["cls"] + "\\" + fn)
    img_slice = img[:,:, img.shape[2]//2, :].astype(float)
    for ch in range(img_slice.shape[-1]):
        img_slice[:, :, ch] *= 255/np.amax(img_slice[:, :, ch])
    img_slice = np.stack([img_slice, img_slice, img_slice], axis=2)
    
    img_slice = draw_bbox(img_slice, C.dims, small_vois[fn[:-4]])
        
    ch1 = np.transpose(img_slice[:,::-1,:,0], (1,0,2))
    ch2 = np.transpose(img_slice[:,::-1,:,1], (1,0,2))
    
    if num_ch == 2:
        ret = np.empty([ch1.shape[0]*2, ch1.shape[1], 3])
        ret[:ch1.shape[0],:,:] = ch1
        ret[ch1.shape[0]:,:,:] = ch2
        
    elif num_ch == 3:
        ch3 = np.transpose(img_slice[:,::-1,:,2], (1,0,2))

        ret = np.empty([ch1.shape[0]*3, ch1.shape[1], 3])
        ret[:ch1.shape[0],:,:] = ch1
        ret[ch1.shape[0]:ch1.shape[0]*2,:,:] = ch2
        ret[ch1.shape[0]*2:,:,:] = ch3
        
    else:
        raise ValueError("Invalid num channels")
        
    imsave("%s\\%s (pred %s).png" % (save_dir, fn[:-4], pred_class), ret)

In [71]:
def condense_cm(y_true, y_pred, cls_mapping):
    simplify_map = {'hcc': 0, 'cyst': 1, 'hemangioma': 1, 'fnh': 1, 'cholangio': 2, 'colorectal': 2}
    y_true_simp = np.array([simplify_map[cls_mapping[y]] for y in y_true])
    y_pred_simp = np.array([simplify_map[cls_mapping[y]] for y in y_pred])
    
    return y_true_simp, y_pred_simp, ['hcc', 'benign', 'malignant non-hcc']

In [26]:
cls_mapping

['hcc', 'hemangioma', 'cyst', 'cholangio', 'colorectal']

In [83]:
#plot_with_bbox(fn_list[2], cls_mapping[wrong_guesses[2]])
Y_pred = model.predict(X_test)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_test])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

#save_output(Z_test, y_pred, cls_mapping)

In [73]:
Y_pred = model.predict(X_train_orig)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_train_orig])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

save_output(Z_train_orig, y_pred, cls_mapping)

In [85]:
simplify_map = {'hcc': 0, 'cyst': 1, 'hemangioma': 1, 'fnh': 1, 'cholangio': 2, 'colorectal': 2}

In [63]:
cls_mapping

['colorectal', 'hemangioma', 'hcc', 'cyst', 'cholangio']

In [33]:
cm

array([[18,  0,  0,  3,  1],
       [ 1, 19,  0,  2,  2],
       [ 1,  3, 26,  0,  2],
       [ 1,  3,  0, 14,  6],
       [ 9,  6,  0,  2, 11]], dtype=int64)

In [86]:
y_true_simp, y_pred_simp, _ = condense_cm(y_true, y_pred, cls_mapping)

In [87]:
accuracy_score(y_true_simp, y_pred_simp)

0.83076923076923082

In [84]:
accuracy_score(y_true, y_pred)

0.73076923076923073

4 cls: 94/94%, 86/88%, 86%/90%
5 cls: 80%/89%, 72%/86%


In [365]:
fn_list = Z_test[~np.equal(y_pred, y_true)]
wrong_guesses = np.array(y_pred)[~np.equal(y_pred, y_true)]

In [409]:
Z_test[~np.equal(y_pred, y_true)]

array(['E100815762_0.npy', '12239513_0.npy', '12042703_2.npy'],
      dtype='<U16')

In [21]:
voi_df_art[voi_df_art["Filename"] == "12673726.npy"]

,Filename,x1,x2,y1,y2,z1,z2,cls,flipz,dx,dy,dz,id
119,12673726.npy,105,134,76,101,40,53,hemangioma,True,29,25,13,126737261057640
120,12673726.npy,127,139,103,117,44,51,hemangioma,True,12,14,7,1267372612710344


In [22]:
voi_df_ven[voi_df_ven["id"] == "1267372612710344"]

,id,x1,x2,y1,y2,z1,z2,dx,dy,dz
74,1267372612710344,127,139,103,117,44,52,12,14,8


activ = model2.predict(X_train)
#activ = model2.predict(np.expand_dims(X_train[10],axis=0))

In [316]:
import operator
#print("Ground truth:", [cls_mapping[max(enumerate(x), key=operator.itemgetter(1))[0]] for x in Y_val[::30]])
Y_ = model.predict(X_test)
print("Predictions:", [cls_mapping[max(enumerate(x), key=operator.itemgetter(1))[0]] + " (%.5f%% confidence)" % (max(x)*100) for x in Y_])

Predictions: ['hcc (95.75861% confidence)', 'hcc (99.98515% confidence)', 'hcc (56.77554% confidence)', 'hcc (97.68384% confidence)', 'hcc (96.73136% confidence)', 'hcc (99.99979% confidence)', 'hcc (99.99988% confidence)', 'hcc (99.71866% confidence)', 'cyst (63.61439% confidence)', 'cyst (51.41591% confidence)', 'hcc (99.65365% confidence)', 'hcc (52.72004% confidence)', 'hcc (57.44457% confidence)', 'cyst (99.82330% confidence)', 'cyst (83.69393% confidence)']


In [86]:
a=time.time()
Y_ = model.predict(X_val)
print(time.time()-a)

0.20054101943969727


In [136]:
import matplotlib.pyplot as plt
%matplotlib inline